In [1]:
import numpy as np
import datetime

np.random.seed(1337)  # for reproducibility

In [3]:
from contextlib import suppress
with suppress(Exception):from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from numpy import nan

now = datetime.datetime.now

In [4]:
now = datetime.datetime.now

batch_size = 128
nb_classes = 5
nb_epoch = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

In [5]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [6]:
def train_model(model, train, test, nb_classes):
    
    X_train = train[0].reshape((train[0].shape[0],) + input_shape)
    X_test = test[0].reshape((test[0].shape[0],) + input_shape)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(train[1], nb_classes)
    Y_test = np_utils.to_categorical(test[1], nb_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(X_train, Y_train,
              batch_size=batch_size, nb_epoch=nb_epoch,
              verbose=1,
              validation_data=(X_test, Y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [7]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
X_train_lt5 = X_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
X_test_lt5 = X_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

X_train_gte5 = X_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
X_test_gte5 = X_test[y_test >= 5]         # np_utils.to_categorical
y_test_gte5 = y_test[y_test >= 5] - 5

11493376/11490434 [==============================] - 0s 0us/step


In [8]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Convolution2D(nb_filters, kernel_size, kernel_size,
                  border_mode='valid',
                  input_shape=input_shape),
    Activation('relu'),
    Convolution2D(nb_filters, kernel_size, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=(pool_size, pool_size)),
    Dropout(0.25),
    Flatten(),
]
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(nb_classes),
    Activation('softmax')
]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1..., padding="valid")`
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  import sys


In [9]:
# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model,
            (X_train_lt5, y_train_lt5),
            (X_test_lt5, y_test_lt5), nb_classes)

X_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples


/opt/conda/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 8s 248us/step - loss: 0.1648 - acc: 0.9489 - val_loss: 0.0278 - val_acc: 0.9903
Epoch 2/5
30596/30596 [==============================] - 4s 116us/step - loss: 0.0498 - acc: 0.9857 - val_loss: 0.0140 - val_acc: 0.9949
Epoch 3/5
30596/30596 [==============================] - 4s 116us/step - loss: 0.0342 - acc: 0.9897 - val_loss: 0.0122 - val_acc: 0.9955
Epoch 4/5
30596/30596 [==============================] - 4s 116us/step - loss: 0.0278 - acc: 0.9921 - val_loss: 0.0080 - val_acc: 0.9973
Epoch 5/5
30596/30596 [==============================] - 4s 116us/step - loss: 0.0227 - acc: 0.9935 - val_loss: 0.0074 - val_acc: 0.9977
Training time: 0:00:22.142799
Test score: 0.007365920290193904
Test accuracy: 0.9976649153531816


In [10]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (X_train_gte5, y_train_gte5),
            (X_test_gte5, y_test_gte5), nb_classes)

X_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples


/opt/conda/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 2s 70us/step - loss: 0.2278 - acc: 0.9335 - val_loss: 0.0475 - val_acc: 0.9840
Epoch 2/5
29404/29404 [==============================] - 2s 63us/step - loss: 0.0790 - acc: 0.9757 - val_loss: 0.0350 - val_acc: 0.9893
Epoch 3/5
29404/29404 [==============================] - 2s 62us/step - loss: 0.0602 - acc: 0.9816 - val_loss: 0.0295 - val_acc: 0.9905
Epoch 4/5
29404/29404 [==============================] - 2s 62us/step - loss: 0.0509 - acc: 0.9845 - val_loss: 0.0269 - val_acc: 0.9897
Epoch 5/5
29404/29404 [==============================] - 2s 62us/step - loss: 0.0454 - acc: 0.9860 - val_loss: 0.0247 - val_acc: 0.9918
Training time: 0:00:09.625290
Test score: 0.02474974749833127
Test accuracy: 0.9917712403751405
